# Notebook Objective and Setup

BGG01 involves the acquisition of game data from BoardGameGeek. Largely this is accomplished by XML API call, with some dynamic content scraped. Files are dumped to a "dirty" directory.

## Package Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from bs4 import BeautifulSoup
import requests
import regex as re
import time
import json
import os
import gc
from io import StringIO, BytesIO
from lxml import etree

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os


## Functions

In [2]:
def create_designers(game_page, game_id):
    '''Create DataFrame for Designers for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all designers on page
    all_designers = game_page.find_all('link', type='boardgamedesigner')
    
    # make dataframe
    designers = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    design = {'BGGId':int(game_id)}
    
    # add this item's designers to dictionary
    for item in all_designers:
        design[item['value']] = int(1)
    
    # append to dataframe
    designers = designers.append(design, ignore_index=True)
    
    # return dataframe
    return designers

In [3]:
def create_categories(game_page, game_id):
    '''Create DataFrame for Categories for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all categories on page
    all_categories = game_page.find_all('link', type='boardgamecategory')
    
    # make dataframe
    categories = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    category = {'BGGId':int(game_id)}
    
    # add this item's categories to dictionary
    for item in all_categories:
        category[item['value']] = int(1)
    
    # append to dataframe
    categories = categories.append(category, ignore_index=True)
    
    # return dataframe
    return categories

In [4]:
def create_mechanics(game_page, game_id):
    '''Create DataFrame for Mechanics for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all mechanics on page
    all_mechanics = game_page.find_all('link', type='boardgamemechanic')
    # make dataframe
    mechanics = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    mechanic = {'BGGId':int(game_id)}

    # add this item's mechanics to dictionary
    for item in all_mechanics:
        mechanic[item['value']] = int(1)
    
    # Try Tableau
    try:
        game_page.find('link', type='boardgamefamily', value=("Mechanism: Tableau Building"))['value']
        mechanic['TableauBuilding'] = int(1)
    except: pass
    
    # Try is Legacy
    try:
        game_page.find('link', type='boardgamefamily', value=("Mechanism: Legacy"))['value']
        mechanic['Legacy'] = int(1)
    except: pass
    
    # append to dataframe
    mechanics = mechanics.append(mechanic, ignore_index=True)
    # return dataframe
    return mechanics

In [5]:
def create_artists(game_page, game_id):
    '''Create DataFrame for artists for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all artists on page
    all_artists = game_page.find_all('link', type='boardgameartist')
    
    # make dataframe
    artists = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    artist = {'BGGId':int(game_id)}

    # add this item's artists to dictionary
    for item in all_artists:
        artist[item['value']] = int(1)
    
    # append to dataframe
    artists = artists.append(artist, ignore_index=True)
    
    # return dataframe
    return artists

In [6]:
def create_publishers(game_page, game_id):
    '''Create DataFrame for Mechanics for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all publishers on page
    all_publishers = game_page.find_all('link', type='boardgamepublisher')
    
    # make dataframe
    publishers = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    publisher = {'BGGId':int(game_id)}
    
    # add this item's artists to dictionary
    for item in all_publishers:
        publisher[item['value']] = int(1)
    
    # append to dataframe
    publishers = publishers.append(publisher, ignore_index=True)
    
    # return dataframe
    return publishers

In [7]:
def create_awards(awards_level, game_id):
    '''Create DataFrame for Awards for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all awards on page
    all_awards = awards_level.find_all('a', class_='ng-binding')
    
    # make dataframe
    awards = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    award = {'BGGId':int(game_id)}

    # add this item's awards to dictionary
    for item in all_awards:
        item = re.sub("[0-9]", "", item.text).strip(' ')
        award[item] = int(1)
    
    # append to dataframe
    awards = awards.append(award, ignore_index=True)
    
    # return dataframe
    return awards

# PULL - Game Data

Last game id: 349161

## Pull Games with Scrapy

In [8]:
game_ids_current = pd.read_pickle('data_dirty/big_game_ids.pkl')
game_ids = list(game_ids_current[0])

In [9]:
game_ids_current.head()

,0
0,174430
1,161936
2,224517
3,167791
4,233078


In [3]:
def generate_raw_urls(game_ids):
    
    start_position =0
    end_position = 1000
    file_suffix = 0
    urls_list = []
    
    while start_position < (len(game_ids)+1):    
    
        ##### File Setup Section #####
    
        # increment file suffix
        file_suffix += 1
        # get file suffix as string
        suffix_str = str(file_suffix)
    
        # print start and end positions
        print("Getting items "+str(start_position+1)+' through '+str(end_position))
    
        # get list of game ids to grab
        #grab_list = game_ids[0][start_position:end_position]
        grab_list = game_ids[start_position:end_position]
    
        # piece together target string of game ids for BGG 
        targets = ''
        for item in grab_list:
            targets += str(item)+','  
    
        # establish path with targets and current page
        path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&stats=1&type=boardgame'
        urls_list.append(path)    

        start_position += 1000
        end_position += 1000
    
    return urls_list

In [4]:
raw_urls = generate_raw_urls(game_ids)
    
with open('data_dirty/scraper_urls_raw.json', 'w') as convert_file:
    convert_file.write(json.dumps(raw_urls))

Getting items 1 through 1000
Getting items 1001 through 2000
Getting items 2001 through 3000
Getting items 3001 through 4000
Getting items 4001 through 5000
Getting items 5001 through 6000
Getting items 6001 through 7000
Getting items 7001 through 8000
Getting items 8001 through 9000
Getting items 9001 through 10000
Getting items 10001 through 11000
Getting items 11001 through 12000
Getting items 12001 through 13000
Getting items 13001 through 14000
Getting items 14001 through 15000
Getting items 15001 through 16000
Getting items 16001 through 17000
Getting items 17001 through 18000
Getting items 18001 through 19000
Getting items 19001 through 20000
Getting items 20001 through 21000
Getting items 21001 through 22000
Getting items 22001 through 23000
Getting items 23001 through 24000
Getting items 24001 through 25000
Getting items 25001 through 26000
Getting items 26001 through 27000
Getting items 27001 through 28000
Getting items 28001 through 29000
Getting items 29001 through 30000
Ge

In [5]:
len(raw_urls)

134

In [7]:
with open('data_dirty/scraper_urls_raw.json') as json_file:
    scraper_urls_raw = json.load(json_file)

In [8]:
import scrapy

!scrapy crawl bgg_raw

2022-01-20 11:21:39 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: bggscraper)
2022-01-20 11:21:39 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.7.10 | packaged by conda-forge | (default, Oct 13 2021, 20:22:05) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 35.0.0, Platform Windows-10-10.0.19041-SP0
2022-01-20 11:21:39 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-01-20 11:21:39 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'bggscraper',
 'COOKIES_ENABLED': False,
 'DOWNLOAD_DELAY': 2,
 'NEWSPIDER_MODULE': 'bggscraper.spiders',
 'SPIDER_MODULES': ['bggscraper.spiders']}
2022-01-20 11:21:39 [scrapy.extensions.telnet] INFO: Telnet Password: 9c06225ec65c527b
2022-01-20 11:21:39 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'sc

## Process files with BS

In [11]:
files = []

for item in os.listdir('data_dirty/pulled_games_raw/'):
    files.append(item)

In [12]:
# set up our columns list
columns = ['BGGId',
                'Name',
               'Description',
                'YearPublished',
                'GameWeight',
                'AvgRating',  
                'BayesAvgRating',
                'StdDev',
                'MinPlayers',
                'MaxPlayers',
                'ComAgeRec',
                'LanguageEase',
                'BestPlayers',
                'GoodPlayers',
                'NumOwned',
                'NumWant',
                'NumWish',
                'NumWeightVotes',
                'MfgPlaytime',
                'ComMinPlaytime',
                'ComMaxPlaytime',
                'MfgAgeRec',
                'NumUserRatings',
                'NumComments',
                'NumAlternates',
                'NumExpansions',
                'NumImplementations',
           'IsReimplementation',
                'Family',
                'Theme',
               'Category',
               'Kickstarted',
               'ImagePath',
          ]

In [13]:
file_suffix = 0

start = time.time()

for file in files:

    games = pd.DataFrame(columns=columns)
    designers = pd.DataFrame(columns=['BGGId'])
    categories = pd.DataFrame(columns=['BGGId'])
    mechanics = pd.DataFrame(columns=['BGGId'])
    artists = pd.DataFrame(columns=['BGGId'])
    publishers = pd.DataFrame(columns=['BGGId'])
    subcategories = pd.DataFrame(columns=['BGGId'])
    comments = pd.DataFrame(columns=['BGGId'])
    
    ##### File Setup Section #####
    
    # increment file suffix
    file_suffix += 1
    # get file suffix as string
    suffix_str = str(file_suffix)

    
    path = 'data_dirty/pulled_games_raw/'+file
    print(path)
    
    game_page = BeautifulSoup(open(path, encoding="utf8"), "lxml") # parse page with beautifulsoup    
    
    # make entry for each game item on page
    game_entries = game_page.find_all('item')
    
    print("Items loaded. Processing.")
    ##### Process Each Game #####
    
    for entry in game_entries:

        # check that this game has sufficient user ratings to incluide
        try:
            user_ratings = int(entry.find('usersrated')['value'])# get the number of user ratings
        
            if user_ratings < 30: #check if user ratings are under 30
                continue
        except: continue
            
        # get game name and BGG ID
        game_name = entry.find('name', type='primary')['value']
        game_id = entry['id']
        print("Name: "+game_name+", BGG ID: "+str(game_id))

        
        ##### Get Basic Stats #####

        #print("Getting basic stats")
        description = entry.find('description').text # description text of the game
        
        try:
            year_pub = int(entry.find('yearpublished')['value']) # year published
            if year_pub > 2021:
                continue
        except: pass
            
        try: minplayers = int(entry.find('minplayers')['value']) # minimum players
        except: minplayers = None
            
        try: maxplayers = int(entry.find('maxplayers')['value']) # maximum players
        except: maxplayers = None
            
        avg_rating = float(entry.find('average')['value']) # average rating
        bayes_avg = float(entry.find('bayesaverage')['value']) # bayes average rating
        std_dev = float(entry.find('stddev')['value']) # standard deviation of rating
        num_owned = int(entry.find('owned')['value']) # num of people own this game
        num_want = int(entry.find('wanting')['value']) # num of people want this game
        num_wish = int(entry.find('wishing')['value']) # num of people with game on wishlist
        num_weight_votes = int(entry.find('numweights')['value']) # num of votes for game weight
        game_weight = float(entry.find('averageweight')['value']) # voted game weight
        
        try: image_path = entry.find('image').text # path to image
        except: image_path = None
            
        try:  mfg_play_time = int(entry.find('playingtime')['value']) # mfg stated playtime
        except: mfg_play_time = None
        try: comm_min_play = int(entry.find('minplaytime')['value']) # community min playtime
        except: comm_min_play = None
            
        try: comm_max_play = int(entry.find('maxplaytime')['value']) # community max playtime
        except: comm_max_play = None
        
        try: mfg_age = int(entry.find('minage')['value']) # mfg min age
        except: mfg_age = None
            
        #num_comments = int(entry.find('comments')['totalitems']) # num of ratings comments
        num_alts = len(entry.find_all('name', type='alternate')) # number alternate versions
        num_expansions = len(entry.find_all('link', type='boardgameexpansion')) # number of expansions
        num_implementations = len(entry.find_all('link', type='boardgameimplementation')) # number of implementations    
        

        
        ##### Get reimplementation flag #####
        reimplementation = entry.find('link', type="boardgameimplementation", inbound="true") # check if game is a reimplementation
        if reimplementation: 
            reimplements = 1 # if it's a reimplementation, flag it 1
        else: 
            reimplements = 0
 


        ##### Basic stats requiring some compaction/refinement #####

        # community age min
        try:
            age_poll = entry.find('poll', title="User Suggested Player Age").find_all('result')

            total = 0
            items = 0
        
            for item in age_poll:   
                vote = int(item['numvotes']) * int(item['value'][:2])
                total += vote
                items += int(item['numvotes'])

            if items>0: comm_age = total/items # make sure not dividing by 0, get community recommended age
            else: comm_age=None # if no votes, record none
        except: comm_age=None
        
        # Language Ease
        try:
         
            lang_poll = entry.find('poll', title="Language Dependence").find_all('result')
            total, items = 0, 0

            for item in lang_poll:   
                vote = int(item['numvotes']) * int(item['level'])
                total += vote
                items += int(item['numvotes'])

            if items>0: lang_ease = total/items # make sure not dividing by 0, get community language ease
            else: lang_ease=None # if no votes, record none
        except: lang_ease=None # if no votes, record none
            
        try:    
            # Best and Good Players
            players = entry.find('poll', title="User Suggested Number of Players").find_all('results') # get user players poll
            player_num_votes = int(entry.find('poll', title="User Suggested Number of Players")['totalvotes'])# get total votes
        
            best_players, best_score, good_players = 0, 0, [] # set up for best players loop
        
            if player_num_votes > 30: # evaluate if more than 30 votes for num players
                for player in players:
                    best = int(player.find('result', value='Best')['numvotes'])
                    rec = int(player.find('result', value='Recommended')['numvotes'])
                    score = best*2 + rec*1
                    positives = best+rec
                    ratio = positives/player_num_votes
                    if score > best_score: best_players, best_score = player['numplayers'], score # put in # players for best score
                    if ratio > .5: good_players.append(player['numplayers']) # put in good players if over 50% ratio
            else: best_players=None
        except: best_players=None           
        
        # make dataframe for this game
        this_game = pd.DataFrame()
        this_game['BGGId']=int(game_id),
        this_game['Name']=game_name,
        this_game['Description']=description,
        this_game['YearPublished']=int(year_pub),
        this_game['GameWeight']=float(game_weight),
        this_game['AvgRating']=float(avg_rating),
        this_game['BayesAvgRating']=float(bayes_avg),
        this_game['StdDev']=float(std_dev),
        this_game['MinPlayers']=minplayers,
        this_game['MaxPlayers']=maxplayers,
        try: this_game['ComAgeRec']=float(comm_age),
        except: this_game['ComAgeRec']=None,
        try: this_game['LanguageEase']=float(lang_ease),
        except: this_game['LanguageEase']=None,
        this_game['BestPlayers']=best_players,
        this_game['GoodPlayers']=good_players,
        this_game['NumOwned']=int(num_owned),
        this_game['NumWant']=int(num_want),
        this_game['NumWish']=int(num_wish),
        this_game['NumWeightVotes']=int(num_weight_votes),
        this_game['MfgPlaytime']=mfg_play_time,
        this_game['ComMinPlaytime']=comm_min_play,
        this_game['ComMaxPlaytime']=comm_max_play,
        this_game['MfgAgeRec']=mfg_age,
        this_game['NumUserRatings']=int(user_ratings),
        #this_game['NumComments']=int(num_comments),
        this_game['NumAlternates']=int(num_alts),
        this_game['NumExpansions']=int(num_expansions),
        this_game['NumImplementations']=int(num_implementations),
        this_game['IsReimplementation']=int(reimplements),
        this_game['ImagePath']=image_path
            
        
        # add unique information to end of df
        
        # Add game ranks
        ranks = entry.find_all('rank')
        try:
            for item in ranks:
                this_game['Rank:'+item['name']] = float(item['value'])
        except: pass
        
        # Try to add components
        try: 
            families = entry.find_all('link', type='boardgamefamily', value=re.compile("Component"))
            for item in families:                    
                this_game['Components:'+item['name']] = item['value']
        except: pass
            
        
        # Try to add game series/family
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Game:"))['value'].strip('Game:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Series:"))['value'].strip('Series:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            setting = entry.find('link', type='boardgamefamily', value=re.compile("Setting:"))['value'].strip('Setting:').strip(' ')
            this_game['Setting'] = setting
        except: pass
            
        
        # Try to add theme
        try:
            theme = entry.find('link', type='boardgamefamily', value=re.compile("Theme:"))['value'].strip('Theme:').strip(' ')
            this_game['Theme'] = theme
        except: pass
            
        try:
            mechanism = entry.find('link', type='boardgamefamily', value=re.compile("Mechanism:"))['value'].strip('Mechanism:').strip(' ')
            this_game['Mechanism'] = mechanism
        except: pass
        
        # Try to add game category
        try:
            category = entry.find('link', type='boardgamefamily', value=re.compile("Category:"))['value'].strip('Category:').strip(' ')
            this_game['Category'] = category
        except: pass
        
        
        # Try is Kickstarted
        try:
            entry.find('link', type='boardgamefamily', value=re.compile("Crowdfunding"))['value']
            this_game['Kickstarted'] = int(1)
        except: pass
     
    
    
        ##### Get subcategories #####
        
        all_subcategories = entry.find_all('link', type='boardgamecategory')
        
        categories_hold = pd.DataFrame(columns=['BGGId'])
        subcategory = {'BGGId':int(game_id)}
    
        for item in all_subcategories:
            subcategory[item['value']] = int(1)      
        
        categories_hold = categories_hold.append(subcategory, ignore_index=True)    
        
        
        # create specialty dataframes
        designer = create_designers(entry, game_id)
        category = create_categories(entry, game_id)
        mechanic = create_mechanics(entry, game_id)
        artist = create_artists(entry, game_id)
        publisher = create_publishers(entry, game_id)
            
        games = games.append(this_game, ignore_index = True)
        designers = designers.append(designer, ignore_index=True)
        categories = categories.append(category, ignore_index=True)
        mechanics = mechanics.append(mechanic, ignore_index=True)
        artists = artists.append(artist, ignore_index=True)
        publishers = publishers.append(publisher, ignore_index=True)
        subcategories = subcategories.append(categories_hold, ignore_index=True)
        
    games.to_pickle('data_dirty/pulled_games_processed/games'+suffix_str+'.pkl')
    designers.to_pickle('data_dirty/pulled_games_processed/designers'+suffix_str+'.pkl')
    categories.to_pickle('data_dirty/pulled_games_processed/categories'+suffix_str+'.pkl')
    mechanics.to_pickle('data_dirty/pulled_games_processed/mechanics'+suffix_str+'.pkl')
    artists.to_pickle('data_dirty/pulled_games_processed/artists'+suffix_str+'.pkl')
    publishers.to_pickle('data_dirty/pulled_games_processed/publishers'+suffix_str+'.pkl')
    subcategories.to_pickle('data_dirty/pulled_games_processed/subcategories'+suffix_str+'.pkl')
    
    print("Finished items in this group")    
        
print(f'Time: {time.time() - start}\n\n') 

data_dirty/pulled_games_raw/raw_1642706500.4058533.xml
Items loaded. Processing.
Name: Gloomhaven, BGG ID: 174430
Name: Pandemic Legacy: Season 1, BGG ID: 161936
Name: Brass: Birmingham, BGG ID: 224517
Name: Terraforming Mars, BGG ID: 167791
Name: Twilight Imperium: Fourth Edition, BGG ID: 233078
Name: Gloomhaven: Jaws of the Lion, BGG ID: 291457
Name: Gaia Project, BGG ID: 220308
Name: Star Wars: Rebellion, BGG ID: 187645
Name: Through the Ages: A New Story of Civilization, BGG ID: 182028
Name: War of the Ring: Second Edition, BGG ID: 115746
Name: Spirit Island, BGG ID: 162886
Name: Great Western Trail, BGG ID: 193738
Name: Twilight Struggle, BGG ID: 12333
Name: Scythe, BGG ID: 169786
Name: The Castles of Burgundy, BGG ID: 84876
Name: Nemesis, BGG ID: 167355
Name: 7 Wonders Duel, BGG ID: 173346
Name: Dune: Imperium, BGG ID: 316554
Name: Concordia, BGG ID: 124361
Name: Terra Mystica, BGG ID: 120677
Name: Brass: Lancashire, BGG ID: 28720
Name: A Feast for Odin, BGG ID: 177736
Name: Wing

Name: Obsession, BGG ID: 231733
Name: Secret Hitler, BGG ID: 188834
Name: YINSH, BGG ID: 7854
Name: Goa, BGG ID: 9216
Name: The Search for Planet X, BGG ID: 279537
Name: Dominion (Second Edition), BGG ID: 209418
Name: The Princes of Florence, BGG ID: 555
Name: Deception: Murder in Hong Kong, BGG ID: 156129
Name: Roll Player, BGG ID: 169426
Name: Century: Golem Edition, BGG ID: 232832
Name: Sushi Go Party!, BGG ID: 192291
Name: Arcadia Quest, BGG ID: 155068
Name: Hanamikoji, BGG ID: 158600
Name: Dune, BGG ID: 283355
Name: Flamme Rouge, BGG ID: 199478
Name: Galaxy Trucker, BGG ID: 31481
Name: Viticulture, BGG ID: 128621
Name: Steam, BGG ID: 27833
Name: Shogun, BGG ID: 20551
Name: Hero Realms, BGG ID: 198994
Name: Isle of Skye: From Chieftain to King, BGG ID: 176494
Name: 1830: Railways & Robber Barons, BGG ID: 421
Name: MicroMacro: Crime City, BGG ID: 318977
Name: Under Falling Skies, BGG ID: 306735
Name: Modern Art, BGG ID: 118
Name: Glory to Rome, BGG ID: 19857
Name: 1960: The Making o

Name: 1775: Rebellion, BGG ID: 128996
Name: Blood Bowl (Third Edition), BGG ID: 712
Name: Mansions of Madness, BGG ID: 83330
Name: Descent: Journeys in the Dark, BGG ID: 17226
Name: Shadows of Brimstone: City of the Ancients, BGG ID: 146791
Name: Mystic Vale, BGG ID: 194607
Name: Forbidden Desert, BGG ID: 136063
Name: Raptor, BGG ID: 177639
Name: Destinies, BGG ID: 285192
Name: Exit: The Game – Dead Man on the Orient Express, BGG ID: 226522
Name: Labyrinth: The War on Terror, 2001 – ?, BGG ID: 62227
Name: Colt Express, BGG ID: 158899
Name: BattleCON: Devastation of Indines, BGG ID: 123123
Name: Deus, BGG ID: 162082
Name: Stuffed Fables, BGG ID: 233312
Name: Pipeline, BGG ID: 256730
Name: Madeira, BGG ID: 95527
Name: Hanabi, BGG ID: 98778
Name: Citadels, BGG ID: 478
Name: Smartphone Inc., BGG ID: 246684
Name: Chess, BGG ID: 171
Name: Fire in the Lake, BGG ID: 144189
Name: Catan, BGG ID: 13
Name: Shadows over Camelot, BGG ID: 15062
Name: Black Orchestra, BGG ID: 156858
Name: Friday, BGG 

Name: The Oracle of Delphi, BGG ID: 193558
Name: D-Day at Omaha Beach, BGG ID: 29603
Name: BANG! The Dice Game, BGG ID: 143741
Name: Wir sind das Volk!, BGG ID: 165401
Name: Dragon Castle, BGG ID: 232219
Name: Prêt-à-Porter, BGG ID: 87890
Name: Mexica, BGG ID: 2955
Name: Century: Eastern Wonders, BGG ID: 242574
Name: Fleet: The Dice Game, BGG ID: 245961
Name: K2, BGG ID: 73761
Name: Onirim (Second Edition), BGG ID: 156336
Name: Ashes Reborn: Rise of the Phoenixborn, BGG ID: 167400
Name: My Little Scythe, BGG ID: 226320
Name: Arcadia Quest: Inferno, BGG ID: 179803
Name: Yspahan, BGG ID: 22345
Name: Eminent Domain, BGG ID: 68425
Name: City of the Big Shoulders, BGG ID: 214880
Name: Whitehall Mystery, BGG ID: 190082
Name: Treasure Island, BGG ID: 242639
Name: Middle-Earth Quest, BGG ID: 31563
Name: Empires: Age of Discovery, BGG ID: 173442
Name: Shipyard, BGG ID: 55600
Name: Ascension: Deckbuilding Game, BGG ID: 69789
Name: Carcassonne: The Castle, BGG ID: 7717
Name: Up Front, BGG ID: 586

Name: Unlock!: Timeless Adventures, BGG ID: 279613
Name: Tiny Epic Quest, BGG ID: 201921
Name: Jaws, BGG ID: 272738
Name: Attika, BGG ID: 8051
Name: Zooloretto, BGG ID: 27588
Name: Dungeons & Dragons: Castle Ravenloft Board Game, BGG ID: 59946
Name: Concept, BGG ID: 147151
Name: The City of Kings, BGG ID: 207243
Name: New York Zoo, BGG ID: 300877
Name: Snow Tails, BGG ID: 38054
Name: Fabled Fruit, BGG ID: 203427
Name: 5-Minute Dungeon, BGG ID: 207830
Name: Flick 'em Up!, BGG ID: 169124
Name: Letter Jam, BGG ID: 275467
Name: Meeple Circus, BGG ID: 193214
Name: Detective: City of Angels, BGG ID: 218074
Name: Parade, BGG ID: 56692
Name: Werewords, BGG ID: 219215
Name: Call to Adventure, BGG ID: 238992
Name: DC Comics Deck-Building Game, BGG ID: 125678
Name: Blue Lagoon, BGG ID: 244331
Name: Ascension: Return of the Fallen, BGG ID: 95064
Name: La Città, BGG ID: 554
Name: The Grimm Forest, BGG ID: 212402
Name: SpaceCorp: 2025-2300AD, BGG ID: 214029
Name: Gold West, BGG ID: 154086
Name: Lege

...

Name: Carnival, BGG ID: 101930
Name: MixUp, BGG ID: 22399
Name: The Great Pacific War: The Second World War in the Pacific, BGG ID: 8402
Name: Rok 1920, BGG ID: 21685
Name: Aliens: Hadley's Hope, BGG ID: 257313
Name: Hogwarts: House Cup Challenge, BGG ID: 37683
Name: Le Temps des As, BGG ID: 161682
Name: BrikWars, BGG ID: 7623
Name: Savage Worlds Showdown: Miniatures Rules for All Genres, BGG ID: 35654
Name: Dice Academy, BGG ID: 271756
Name: Star Wars Labyrinth, BGG ID: 130600
Name: Kriegbot, BGG ID: 38266
Name: Kasserine Pass, BGG ID: 8736
Name: Hetzer Sniper!: Sniper Companion Game #1, BGG ID: 6470
Name: Lumeria: War of the Gods, BGG ID: 284987
Name: Duel for Kharkov, BGG ID: 14387
Name: Attangle, BGG ID: 24525
Name: Epées et Hallebardes 1315-1476, BGG ID: 37592
Name: Rescuing Robin Hood, BGG ID: 313269
Name: Rice Dice, BGG ID: 247034
Name: Gallipoli, 1915: Churchill's Greatest Gamble, BGG ID: 163319
Name: The Grunwald Swords, BGG ID: 206464
Name: Exhibition: 20th Century, BGG ID: 2

Items loaded. Processing.
Name: New Rules for Classic Games, BGG ID: 10904
Name: Dice Games Properly Explained, BGG ID: 10973
Name: Four Battles in North Africa, BGG ID: 11054
Finished items in this group
data_dirty/pulled_games_raw/raw_1642708149.9814823.xml
Items loaded. Processing.
Finished items in this group
data_dirty/pulled_games_raw/raw_1642708203.826162.xml
Items loaded. Processing.
Name: BANG! The Bullet!, BGG ID: 30933
Name: Tempête sur l'Échiquier, BGG ID: 30966
Finished items in this group
data_dirty/pulled_games_raw/raw_1642708207.159608.xml
Items loaded. Processing.
Name: War of the Rebellion, BGG ID: 14590
Name: International Cricket, BGG ID: 15238
Finished items in this group
data_dirty/pulled_games_raw/raw_1642708303.0006437.xml
Items loaded. Processing.
Finished items in this group
data_dirty/pulled_games_raw/raw_1642708334.9354777.xml
Items loaded. Processing.
Name: Mü and Lots More, BGG ID: 32928
Name: Trivial Pursuit: Greatest Hits, BGG ID: 33023
Finished items in

Name: Mystery of the Abbey with The Pilgrims' Chronicles, BGG ID: 153912
Name: Wanted! Dead or Alive, BGG ID: 153967
Name: Carcassonne Limited Edition, BGG ID: 154094
Finished items in this group
data_dirty/pulled_games_raw/raw_1642712917.1542323.xml
Items loaded. Processing.
Name: Puerto Rico + Expansion I, BGG ID: 165332
Name: Council of Verona (Second Edition), BGG ID: 165469
Name: The Majority: Complete Edition, BGG ID: 165492
Name: Bohnanza: Ladies & Gangsters, BGG ID: 168421
Finished items in this group
data_dirty/pulled_games_raw/raw_1642712961.7141922.xml
Items loaded. Processing.
Name: The Werewolves of Miller's Hollow: The Pact, BGG ID: 168680
Name: AEG Black Friday Black Box, BGG ID: 169814
Name: Lancaster: Big Box, BGG ID: 170815
Name: Munchkin Legends Deluxe, BGG ID: 171199
Name: Orléans: Deluxe Edition, BGG ID: 171905
Name: El Grande Big Box, BGG ID: 171908
Finished items in this group
data_dirty/pulled_games_raw/raw_1642713113.7801.xml
Items loaded. Processing.
Name: Fal

data_dirty/pulled_games_raw/raw_1642714332.6946714.xml
Items loaded. Processing.
Name: Adeptus Titanicus: The Horus Heresy Rules Set, BGG ID: 259134
Name: Quarriors! Qultimate Quedition, BGG ID: 259571
Name: Feudum: Big Box, BGG ID: 259586
Name: Escape Room: The Game (Escape Rooms II), BGG ID: 260000
Name: New Salem: Second Edition, BGG ID: 260126
Name: Stone Age: Anniversary, BGG ID: 260678
Name: Mississippi Queen, BGG ID: 260927
Name: Ascension: Year Five Collector's Edition, BGG ID: 261588
Finished items in this group
data_dirty/pulled_games_raw/raw_1642714337.5024846.xml
Items loaded. Processing.
Name: AEG Black Friday Black Box 2015, BGG ID: 188818
Name: Munchkin 1+2, BGG ID: 190553
Name: Splendor: AsmOPlay Kit, BGG ID: 191361
Name: Catan: Big Box, BGG ID: 191710
Name: Cat Tower: Kickstarter Edition, BGG ID: 192467
Name: Fleet Commander: Genesis, BGG ID: 192663
Name: Lobo: 10° Aniversário, BGG ID: 192814
Finished items in this group
data_dirty/pulled_games_raw/raw_1642714404.78158

Items loaded. Processing.
Name: Chocolate Factory: Deluxe Edition, BGG ID: 329434
Name: Ultimate Railroads, BGG ID: 329591
Name: Ticket to Ride: Europe – 15th Anniversary, BGG ID: 329841
Name: Carcassonne: 20th Anniversary Edition, BGG ID: 329954
Name: Meeples & Monsters: Kickstarter Edition, BGG ID: 331190
Name: Sleeping Gods: Kickstarter Edition, BGG ID: 331259
Name: Steampunk Rally Fusion: Atomic Edition, BGG ID: 331992
Finished items in this group
data_dirty/pulled_games_raw/raw_1642715461.9406002.xml
Items loaded. Processing.
Name: Alhambra: Big Box (Second Edition), BGG ID: 326355
Name: The Quacks of Quedlinburg: Big Box, BGG ID: 326869
Name: Escape Room: The Game 3, BGG ID: 327107
Name: Fossilis: Kickstarter Edition, BGG ID: 327242
Name: Fleet: The Dice Game (Second Edition), BGG ID: 328326
Name: Chai (Deluxe Edition), BGG ID: 328636
Finished items in this group
data_dirty/pulled_games_raw/raw_1642715473.074642.xml
Items loaded. Processing.
Name: Everdell: The Complete Collectio

### Data Validation

In [ ]:
subcategories1 = pd.read_pickle('data_dirty/pulled_games/subcategories52.pkl')
games1 = pd.read_pickle('data_dirty/pulled_games/games52.pkl')
designers1 = pd.read_pickle('data_dirty/pulled_games/designers52.pkl')
categories1 = pd.read_pickle('data_dirty/pulled_games/categories52.pkl')
mechanics1 = pd.read_pickle('data_dirty/pulled_games/mechanics52.pkl')
artists1 = pd.read_pickle('data_dirty/pulled_games/artists52.pkl')
publishers1 = pd.read_pickle('data_dirty/pulled_games/publishers52.pkl')

In [ ]:
subcategories1

In [ ]:
games1

In [ ]:
designers1

In [ ]:
categories1

In [ ]:
mechanics1

In [ ]:
artists1

In [ ]:
publishers1

## Combine Files

In [ ]:
#games = pd.DataFrame(columns=columns)
#designers = pd.DataFrame(columns=['BGGId'])
#categories = pd.DataFrame(columns=['BGGId'])
#mechanics = pd.DataFrame(columns=['BGGId'])
#artists = pd.DataFrame(columns=['BGGId'])
#publishers = pd.DataFrame(columns=['BGGId'])
#subcategories = pd.DataFrame(columns=['BGGId'])

In [ ]:
for number in range(1, 354):
    print(number)
    
    #this_games = pd.read_pickle('data_dirty/pulled_games/games'+str(number)+'.pkl')
    #this_designers = pd.read_pickle('data_dirty/pulled_games/designers'+str(number)+'.pkl')
    #this_categories = pd.read_pickle('data_dirty/pulled_games/categories'+str(number)+'.pkl')
    #this_mechanics = pd.read_pickle('data_dirty/pulled_games/mechanics'+str(number)+'.pkl')
    #this_artists = pd.read_pickle('data_dirty/pulled_games/artists'+str(number)+'.pkl')
    #this_publishers = pd.read_pickle('data_dirty/pulled_games/publishers'+str(number)+'.pkl')
    #this_subcategories = pd.read_pickle('data_dirty/pulled_games/subcategories'+str(number)+'.pkl')
    
    #games = games.append(this_games)
    #designers = designers.append(this_designers)
    #categories = categories.append(this_categories)
    #mechanics = mechanics.append(this_mechanics)
    #artists = artists.append(this_artists)
    #publishers = publishers.append(this_publishers)
    #subcategories = subcategories.append(this_subcategories)
    
    

In [ ]:
#games = games.reset_index(drop=True)
#designers = designers.reset_index(drop=True)
#categories = categories.reset_index(drop=True)
#mechanics = mechanics.reset_index(drop=True)
#artists = artists.reset_index(drop=True)
#publishers = publishers.reset_index(drop=True)
#subcategories = subcategories.reset_index(drop=True)

In [ ]:
#games.to_pickle('data_dirty/games.pkl')
#designers.to_pickle('data_dirty/designers.pkl')
#categories.to_pickle('data_dirty/categories.pkl')
#mechanics.to_pickle('data_dirty/mechanics.pkl')
#artists.to_pickle('data_dirty/artists.pkl')
#publishers.to_pickle('data_dirty/publishers.pkl')
#subcategories.to_pickle('data_dirty/subcategories.pkl')

### Data Validation

In [ ]:
games=pd.read_pickle('data_dirty/games.pkl')
designers=pd.read_pickle('data_dirty/designers.pkl')
categories=pd.read_pickle('data_dirty/categories.pkl')
mechanics=pd.read_pickle('data_dirty/mechanics.pkl')
artists=pd.read_pickle('data_dirty/artists.pkl')
publishers=pd.read_pickle('data_dirty/publishers.pkl')
subcategories=pd.read_pickle('data_dirty/subcategories.pkl')

In [ ]:
games

In [ ]:
designers.tail()

In [ ]:
categories.tail()

In [ ]:
mechanics.tail()

In [ ]:
artists.tail()

In [ ]:
publishers.tail()

In [ ]:
subcategories.tail()

In [ ]:
break

# PULL - User Ratings

## Create Scraper URLs

In [9]:
game_ids_current = pd.read_pickle('data_cleaned/game_ids_current.pkl')
game_ids = list(game_ids_current)

games = pd.read_pickle('data_cleaned/games.pkl')

In [10]:
ratings_totals = pd.DataFrame(games['BGGId'])
ratings_totals['RatingsPages'] = np.ceil(games['NumUserRatings']/100).astype('int')
ratings_totals = ratings_totals.sort_values('RatingsPages', ascending=False).reset_index(drop=True)

In [12]:
ratings_totals[:500].head(30)

,BGGId,RatingsPages
0,30549,1082
1,822,1080
2,13,1072
3,68448,893
4,36218,812
5,9209,756
6,178900,736
7,167791,731
8,173346,683
9,31260,659


In [ ]:
group1 = ratings_totals[:1000]
group2 = ratings_totals[1000:2000]
group3 = ratings_totals[2000:3000]
group4 = ratings_totals[3000:4000]
group5 = ratings_totals[4000:5000]
group6 = ratings_totals[5000:6000]
group7 = ratings_totals[6000:7000]
group8 = ratings_totals[7000:8000]
group9 = ratings_totals[8000:9000]
group10 = ratings_totals[9000:10000]
group11 = ratings_totals[10000:11000]
group12 = ratings_totals[11000:12000]
group13 = ratings_totals[12000:13000]
group14 = ratings_totals[13000:14000]
group15 = ratings_totals[14000:15000]
group16 = ratings_totals[15000:16000]
group17 = ratings_totals[16000:17000]
group18 = ratings_totals[17000:18000]
group19 = ratings_totals[18000:19000]
group20 = ratings_totals[19000:20000]
group21 = ratings_totals[20000:21000]
group22 = ratings_totals[21000:]

In [ ]:
groups = [group1, group2, group3, group4, group5, group6, group7, group8, group9, group10,
         group11, group12, group13, group14, group15, group16, group17, group18, group19, group20,
         group21, group22]

In [ ]:
def generate_ratings_urls(group):
    urls_list = []
    
    max_pages = group.RatingsPages.max()
    min_pages = group.RatingsPages.min()
    pages = np.arange(max_pages,0, -1)
    max_size = group.shape[0]-1
    #print(max_pages, max_size)
    
    targets = ''
    current_index = 0
    
    for page in pages:
        
        current_thresh = page
        
        while group.iloc[current_index]['RatingsPages'] == current_thresh:
            
            # get BGGId for the index
            current_item = group.iloc[current_index]['BGGId']            
            
            # target is the current item
            target = str(current_item)   
            # add the target to the targets list
            targets += target+','            
            
            current_index += 1
            if current_index == 999 or current_index == max_size:
                # get BGGId for the index
                current_item = group.iloc[current_index]['BGGId']            
            
                # target is the current item
                target = str(current_item)   
                # add the target to the targets list
                targets += target+','
                break
        
        #print(current_index)
        
        # establish path with targets and current page
        path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&ratingcomments=1&page='+str(page)+'&pagesize=100'
        urls_list.append(path)
    
    return urls_list

In [ ]:
group_urls = {}
group_num = 0

for group in groups:
    group_num += 1
    print(group_num)
    group_urls["group"+str(group_num)] = generate_ratings_urls(group)
    
with open('data_cleaned/scraper_urls_ratings.json', 'w') as convert_file:
    convert_file.write(json.dumps(group_urls))

## Scrape URLs

In [ ]:
import scrapy

for item in group_urls:
    
    print(item)
    
    !scrapy crawl bgg_ratings -a group=$item

## Process files with lxml

### One File Test

In [2]:
path = 'data_dirty/pulled_ratings/ratings_group16_1641776554.0417554.xml'

tree = etree.parse(path)
root = tree.getroot()

# set up empty list to store the ratings found on this page
bggid, names, ratings, user_comments, usernames = [], [], [], [], []

for child in root:
    
    # gets BGGId
    game_id = child.get('id')
    #print(game_id)
    
    # gets game name
    name_line = child.find('name')
    game_name = name_line.attrib.get('value')
    #print(game_name)
    
    # get ratings sections
    comments = child.findall(".//comment")
    
    for comment in comments:
        
        # gets username for comment/rating
        username = comment.get('username')
        #print(username)
        
        # gets user's rating
        rating = comment.get('rating')
        
        # gets user comment text
        comment_text = comment.get('value')
        
        bggid.append(game_id)
        names.append(game_name)
        ratings.append(rating)
        user_comments.append(comment_text)
        usernames.append(username)
    
    # dictionary of lists 
    dict = {'BGGId': bggid, 'Name': names, 'Username': usernames, 'Rating': ratings, 'Comments': user_comments} 
    
    df = pd.DataFrame(dict)

In [3]:
df

,BGGId,Name,Username,Rating,Comments
0,200672,Vs System 2PCG: The Alien Battles,Mr EB Slugworth,10,
1,200672,Vs System 2PCG: The Alien Battles,LogicBalm,10,
2,200672,Vs System 2PCG: The Alien Battles,rebelhr,10,
3,200672,Vs System 2PCG: The Alien Battles,ChadSDaniel,10,
4,200672,Vs System 2PCG: The Alien Battles,Selpoivre,10,
...,...,...,...,...,...
54219,9420,Hear 'N Seek,AnnC,4,"05+, 2-6 players, 20 mins. Tried this with 3 b..."
54220,9420,Hear 'N Seek,sky1ark,4,
54221,9420,Hear 'N Seek,Missu,4,
54222,9420,Hear 'N Seek,Hessu68,4,


### All Files

In [4]:
files = []

for item in os.listdir('data_dirty/pulled_ratings/'):
    files.append(item)

In [5]:
len(files)

1190

In [6]:
raw_ratings = pd.DataFrame()

In [7]:
for file in files:
    
    path = 'data_dirty/pulled_ratings/'+file
    print(path)

    tree = etree.parse(path)
    root = tree.getroot()
    
    # set up empty list to store the ratings found on this page
    bggid, names, ratings, user_comments, usernames = [], [], [], [], []

    for child in root:
    
        # gets BGGId
        game_id = child.get('id')
        #print(game_id)
    
        # gets game name
        name_line = child.find('name')
        game_name = name_line.attrib.get('value')
        #print(game_name)
    
        # get ratings sections
        comments = child.findall(".//comment")
    
        for comment in comments:
        
            # gets username for comment/rating
            username = comment.get('username')
            #print(username)
        
            # gets user's rating
            rating = comment.get('rating')
        
            # gets user comment text
            comment_text = comment.get('value')
        
            bggid.append(game_id)
            names.append(game_name)
            ratings.append(rating)
            user_comments.append(comment_text)
            usernames.append(username)
    
    # dictionary of lists 
    file_dict = {'BGGId': bggid, 'Name': names, 'Username': usernames, 'Rating': ratings, 'Comments': user_comments} 
    
    df = pd.DataFrame(file_dict)
        
    raw_ratings = raw_ratings.append(df)
        
    raw_ratings.to_pickle('data_dirty/raw_game_ratings.pkl')

data_dirty/pulled_ratings/ratings_group10_1641810061.7020946.xml
data_dirty/pulled_ratings/ratings_group10_1641810075.360559.xml
data_dirty/pulled_ratings/ratings_group11_1641810125.5646534.xml
data_dirty/pulled_ratings/ratings_group11_1641810185.951657.xml
data_dirty/pulled_ratings/ratings_group12_1641810232.5131326.xml
data_dirty/pulled_ratings/ratings_group12_1641810237.194839.xml
data_dirty/pulled_ratings/ratings_group13_1641810247.6621652.xml
data_dirty/pulled_ratings/ratings_group13_1641810297.2223392.xml
data_dirty/pulled_ratings/ratings_group14_1641810357.4829957.xml
data_dirty/pulled_ratings/ratings_group15_1641810363.8206854.xml
data_dirty/pulled_ratings/ratings_group16_1641776554.0417554.xml
data_dirty/pulled_ratings/ratings_group16_1641810374.5657291.xml
data_dirty/pulled_ratings/ratings_group17_1641776466.368078.xml
data_dirty/pulled_ratings/ratings_group17_1641810389.2540581.xml
data_dirty/pulled_ratings/ratings_group18_1641776409.8524408.xml
data_dirty/pulled_ratings/rat

data_dirty/pulled_ratings/ratings_group1_1641777643.0850637.xml
data_dirty/pulled_ratings/ratings_group1_1641777646.1145713.xml
data_dirty/pulled_ratings/ratings_group1_1641777651.574091.xml
data_dirty/pulled_ratings/ratings_group1_1641777657.465971.xml
data_dirty/pulled_ratings/ratings_group1_1641777662.7667174.xml
data_dirty/pulled_ratings/ratings_group1_1641777668.5338187.xml
data_dirty/pulled_ratings/ratings_group1_1641777674.6715596.xml
data_dirty/pulled_ratings/ratings_group1_1641777679.974401.xml
data_dirty/pulled_ratings/ratings_group1_1641777683.8055406.xml
data_dirty/pulled_ratings/ratings_group1_1641777688.4003778.xml
data_dirty/pulled_ratings/ratings_group1_1641777693.6137397.xml
data_dirty/pulled_ratings/ratings_group1_1641777697.978359.xml
data_dirty/pulled_ratings/ratings_group1_1641777702.4491167.xml
data_dirty/pulled_ratings/ratings_group1_1641777708.2819104.xml
data_dirty/pulled_ratings/ratings_group1_1641777714.0342228.xml
data_dirty/pulled_ratings/ratings_group1_164

data_dirty/pulled_ratings/ratings_group1_1641778369.5997865.xml
data_dirty/pulled_ratings/ratings_group1_1641778378.9087644.xml
data_dirty/pulled_ratings/ratings_group1_1641778387.12575.xml
data_dirty/pulled_ratings/ratings_group1_1641778394.7001493.xml
data_dirty/pulled_ratings/ratings_group1_1641778400.2872355.xml
data_dirty/pulled_ratings/ratings_group1_1641778408.2139556.xml
data_dirty/pulled_ratings/ratings_group1_1641778415.5561464.xml
data_dirty/pulled_ratings/ratings_group1_1641778422.0360456.xml
data_dirty/pulled_ratings/ratings_group1_1641778427.9169214.xml
data_dirty/pulled_ratings/ratings_group1_1641778435.1610208.xml
data_dirty/pulled_ratings/ratings_group1_1641778439.6631193.xml
data_dirty/pulled_ratings/ratings_group1_1641778445.4999373.xml
data_dirty/pulled_ratings/ratings_group1_1641778452.0128663.xml
data_dirty/pulled_ratings/ratings_group1_1641778457.6194746.xml
data_dirty/pulled_ratings/ratings_group1_1641778462.3297627.xml
data_dirty/pulled_ratings/ratings_group1_1

data_dirty/pulled_ratings/ratings_group1_1641779404.6643846.xml
data_dirty/pulled_ratings/ratings_group1_1641779416.8039415.xml
data_dirty/pulled_ratings/ratings_group1_1641779428.167792.xml
data_dirty/pulled_ratings/ratings_group1_1641779438.6628509.xml
data_dirty/pulled_ratings/ratings_group1_1641779444.8279686.xml
data_dirty/pulled_ratings/ratings_group1_1641779457.3268511.xml
data_dirty/pulled_ratings/ratings_group1_1641779470.6264632.xml
data_dirty/pulled_ratings/ratings_group1_1641779483.3815796.xml
data_dirty/pulled_ratings/ratings_group1_1641779496.2663178.xml
data_dirty/pulled_ratings/ratings_group1_1641779505.5707884.xml
data_dirty/pulled_ratings/ratings_group1_1641779512.9009664.xml
data_dirty/pulled_ratings/ratings_group1_1641779525.4528978.xml
data_dirty/pulled_ratings/ratings_group1_1641779535.0036004.xml
data_dirty/pulled_ratings/ratings_group1_1641779547.4043932.xml
data_dirty/pulled_ratings/ratings_group1_1641779561.2620184.xml
data_dirty/pulled_ratings/ratings_group1_

data_dirty/pulled_ratings/ratings_group1_1641781281.863422.xml
data_dirty/pulled_ratings/ratings_group1_1641781295.8372118.xml
data_dirty/pulled_ratings/ratings_group1_1641781317.1281524.xml
data_dirty/pulled_ratings/ratings_group1_1641781344.0138626.xml
data_dirty/pulled_ratings/ratings_group1_1641781363.3386207.xml
data_dirty/pulled_ratings/ratings_group1_1641781386.9281397.xml
data_dirty/pulled_ratings/ratings_group1_1641781399.8597534.xml
data_dirty/pulled_ratings/ratings_group1_1641781427.5273614.xml
data_dirty/pulled_ratings/ratings_group1_1641781439.2935617.xml
data_dirty/pulled_ratings/ratings_group1_1641781462.4263833.xml
data_dirty/pulled_ratings/ratings_group1_1641781483.544571.xml
data_dirty/pulled_ratings/ratings_group1_1641781506.8273857.xml
data_dirty/pulled_ratings/ratings_group1_1641781516.846251.xml
data_dirty/pulled_ratings/ratings_group1_1641781538.2639709.xml
data_dirty/pulled_ratings/ratings_group1_1641781550.8424072.xml
data_dirty/pulled_ratings/ratings_group1_16

data_dirty/pulled_ratings/ratings_group1_1641784287.0500107.xml
data_dirty/pulled_ratings/ratings_group1_1641784303.7820265.xml
data_dirty/pulled_ratings/ratings_group1_1641784328.6482632.xml
data_dirty/pulled_ratings/ratings_group1_1641784358.3572397.xml
data_dirty/pulled_ratings/ratings_group1_1641784389.2275295.xml
data_dirty/pulled_ratings/ratings_group1_1641784411.3687458.xml
data_dirty/pulled_ratings/ratings_group1_1641784442.597894.xml
data_dirty/pulled_ratings/ratings_group1_1641784474.0185242.xml
data_dirty/pulled_ratings/ratings_group1_1641784496.9316926.xml
data_dirty/pulled_ratings/ratings_group1_1641784530.1396003.xml
data_dirty/pulled_ratings/ratings_group1_1641784562.1900704.xml
data_dirty/pulled_ratings/ratings_group1_1641784587.478052.xml
data_dirty/pulled_ratings/ratings_group1_1641784616.5271769.xml
data_dirty/pulled_ratings/ratings_group1_1641784638.1788156.xml
data_dirty/pulled_ratings/ratings_group1_1641784664.6601098.xml
data_dirty/pulled_ratings/ratings_group1_1

data_dirty/pulled_ratings/ratings_group1_1641788824.7728212.xml
data_dirty/pulled_ratings/ratings_group1_1641788856.7915778.xml
data_dirty/pulled_ratings/ratings_group1_1641788909.4073868.xml
data_dirty/pulled_ratings/ratings_group1_1641788949.311172.xml
data_dirty/pulled_ratings/ratings_group1_1641788998.5687551.xml
data_dirty/pulled_ratings/ratings_group1_1641789051.2520192.xml
data_dirty/pulled_ratings/ratings_group1_1641789103.9117527.xml
data_dirty/pulled_ratings/ratings_group1_1641789150.422571.xml
data_dirty/pulled_ratings/ratings_group1_1641789198.82838.xml
data_dirty/pulled_ratings/ratings_group1_1641789249.6295178.xml
data_dirty/pulled_ratings/ratings_group1_1641789304.5767615.xml
data_dirty/pulled_ratings/ratings_group1_1641789357.6516147.xml
data_dirty/pulled_ratings/ratings_group1_1641789415.4378908.xml
data_dirty/pulled_ratings/ratings_group1_1641789472.2454607.xml
data_dirty/pulled_ratings/ratings_group1_1641789513.7426658.xml
data_dirty/pulled_ratings/ratings_group1_164

data_dirty/pulled_ratings/ratings_group1_1641797447.8902912.xml
data_dirty/pulled_ratings/ratings_group1_1641797449.260364.xml
data_dirty/pulled_ratings/ratings_group1_1641797451.9105308.xml
data_dirty/pulled_ratings/ratings_group1_1641797453.8306453.xml
data_dirty/pulled_ratings/ratings_group1_1641797456.1000915.xml
data_dirty/pulled_ratings/ratings_group1_1641798090.1599908.xml
data_dirty/pulled_ratings/ratings_group1_1641798159.2862587.xml
data_dirty/pulled_ratings/ratings_group1_1641798242.2239149.xml
data_dirty/pulled_ratings/ratings_group1_1641798266.3265142.xml
data_dirty/pulled_ratings/ratings_group1_1641798287.2315285.xml
data_dirty/pulled_ratings/ratings_group1_1641798297.4421399.xml
data_dirty/pulled_ratings/ratings_group1_1641798302.512445.xml
data_dirty/pulled_ratings/ratings_group1_1641798305.4226198.xml
data_dirty/pulled_ratings/ratings_group1_1641798307.562749.xml
data_dirty/pulled_ratings/ratings_group1_1641798309.9913847.xml
data_dirty/pulled_ratings/ratings_group1_16

data_dirty/pulled_ratings/ratings_group1_1641803868.9757502.xml
data_dirty/pulled_ratings/ratings_group1_1641803873.0000777.xml
data_dirty/pulled_ratings/ratings_group1_1641803875.5754995.xml
data_dirty/pulled_ratings/ratings_group1_1641803877.891061.xml
data_dirty/pulled_ratings/ratings_group1_1641803901.3512247.xml
data_dirty/pulled_ratings/ratings_group1_1641803904.4614353.xml
data_dirty/pulled_ratings/ratings_group1_1641803910.032597.xml
data_dirty/pulled_ratings/ratings_group1_1641803910.9426503.xml
data_dirty/pulled_ratings/ratings_group1_1641803934.1171556.xml
data_dirty/pulled_ratings/ratings_group1_1641803989.5716946.xml
data_dirty/pulled_ratings/ratings_group1_1641804031.8028438.xml
data_dirty/pulled_ratings/ratings_group1_1641804068.9369514.xml
data_dirty/pulled_ratings/ratings_group1_1641804102.0891218.xml
data_dirty/pulled_ratings/ratings_group1_1641804125.6305487.xml
data_dirty/pulled_ratings/ratings_group1_1641804161.370937.xml
data_dirty/pulled_ratings/ratings_group1_16

data_dirty/pulled_ratings/ratings_group4_1641809488.4746885.xml
data_dirty/pulled_ratings/ratings_group4_1641809492.1951973.xml
data_dirty/pulled_ratings/ratings_group4_1641809498.0005283.xml
data_dirty/pulled_ratings/ratings_group4_1641809507.4349523.xml
data_dirty/pulled_ratings/ratings_group4_1641809520.4957385.xml
data_dirty/pulled_ratings/ratings_group4_1641809525.6765788.xml
data_dirty/pulled_ratings/ratings_group4_1641809535.5500898.xml
data_dirty/pulled_ratings/ratings_group5_1641809537.9451158.xml
data_dirty/pulled_ratings/ratings_group5_1641809547.4735785.xml
data_dirty/pulled_ratings/ratings_group5_1641809559.4344568.xml
data_dirty/pulled_ratings/ratings_group5_1641809561.3120747.xml
data_dirty/pulled_ratings/ratings_group5_1641809562.5655704.xml
data_dirty/pulled_ratings/ratings_group5_1641809583.5104005.xml
data_dirty/pulled_ratings/ratings_group5_1641809637.026339.xml
data_dirty/pulled_ratings/ratings_group6_1641809642.1574228.xml
data_dirty/pulled_ratings/ratings_group6_

In [8]:
raw_ratings.head()

,BGGId,Name,Username,Rating,Comments
0,213788,Ascension: Year Three Collector's Edition,Tonydorrf,8,
1,213788,Ascension: Year Three Collector's Edition,tachyon14k,8,
2,213788,Ascension: Year Three Collector's Edition,Ungotter,8,
3,213788,Ascension: Year Three Collector's Edition,brainlocki3,8,
4,213788,Ascension: Year Three Collector's Edition,PPMP,8,


In [9]:
raw_ratings['BGGId'] = raw_ratings['BGGId'].astype(int)
raw_ratings['Rating'] = raw_ratings['Rating'].astype(float)

In [ ]:
raw_ratings.to_pickle('data_dirty/raw_game_ratings.pkl')

In [10]:
raw_ratings.drop_duplicates(keep='first', inplace=True)

In [11]:
raw_ratings

,BGGId,Name,Username,Rating,Comments
0,213788,Ascension: Year Three Collector's Edition,Tonydorrf,8.0,
1,213788,Ascension: Year Three Collector's Edition,tachyon14k,8.0,
2,213788,Ascension: Year Three Collector's Edition,Ungotter,8.0,
3,213788,Ascension: Year Three Collector's Edition,brainlocki3,8.0,
4,213788,Ascension: Year Three Collector's Edition,PPMP,8.0,
...,...,...,...,...,...
62415,165521,Rory's Story Cubes: Medic,rseater,3.0,"more of the same, which means good dice and a ..."
62416,165521,Rory's Story Cubes: Medic,Bluefox86,3.0,
62417,165521,Rory's Story Cubes: Medic,serginator,3.0,
62418,193488,Airlines,CaptainCattan,1.0,Not delivered - publisher refuses to communica...


# Appendix

## Get Game ids

In [10]:
soup = BeautifulSoup(requests.get("https://www.boardgamegeek.com/browse/boardgame/").content, "html.parser")

In [15]:
last_page = int(soup.find(title="last page").contents[0].strip('[]'))
last_page

1529

In [16]:
game_ids = []

for i in range(1,last_page+1):
    path = "https://boardgamegeek.com/browse/boardgame/page/"+str(i)
    print(path)
    page = requests.get(path)
    rank_titles = BeautifulSoup(page.content, 'html.parser')
    titles = rank_titles.find_all('a', class_='primary')
    print(titles)
    for item in titles:
        string = item['href']
        string = re.sub('[a-z]', '', string)
        string = string.lstrip('//')
        string = string.split('/')[0]
        game_ids.append(int(string))
    time.sleep(1)

https://boardgamegeek.com/browse/boardgame/page/1
[<a class="primary" href="/boardgame/224517/brass-birmingham">Brass: Birmingham</a>, <a class="primary" href="/boardgame/161936/pandemic-legacy-season-1">Pandemic Legacy: Season 1</a>, <a class="primary" href="/boardgame/174430/gloomhaven">Gloomhaven</a>, <a class="primary" href="/boardgame/342942/ark-nova">Ark Nova</a>, <a class="primary" href="/boardgame/233078/twilight-imperium-fourth-edition">Twilight Imperium: Fourth Edition</a>, <a class="primary" href="/boardgame/316554/dune-imperium">Dune: Imperium</a>, <a class="primary" href="/boardgame/167791/terraforming-mars">Terraforming Mars</a>, <a class="primary" href="/boardgame/115746/war-ring-second-edition">War of the Ring: Second Edition</a>, <a class="primary" href="/boardgame/291457/gloomhaven-jaws-lion">Gloomhaven: Jaws of the Lion</a>, <a class="primary" href="/boardgame/187645/star-wars-rebellion">Star Wars: Rebellion</a>, <a class="primary" href="/boardgame/162886/spirit-isla

KeyboardInterrupt: 

In [13]:
len(game_ids)

1000

In [26]:
game_ids = pd.DataFrame(game_ids)
game_ids.to_pickle('data_dirty/big_game_ids.pkl')

In [14]:
list(game_ids[0])[-10:]

TypeError: 'int' object is not iterable

## DEPRECATED

### Pull Games with Selenium/BS

In [9]:
# set up our columns list
columns = ['BGGId',
                'Name',
               'Description',
                'YearPublished',
                'GameWeight',
                'AvgRating',  
                'BayesAvgRating',
                'StdDev',
                'MinPlayers',
                'MaxPlayers',
                'ComAgeRec',
                'LanguageEase',
                'BestPlayers',
                'GoodPlayers',
                'NumOwned',
                'NumWant',
                'NumWish',
                'NumWeightVotes',
                'MfgPlaytime',
                'ComMinPlaytime',
                'ComMaxPlaytime',
                'MfgAgeRec',
                'NumUserRatings',
                'NumComments',
                'NumAlternates',
                'NumExpansions',
                'NumImplementations',
           'IsReimplementation',
                'Family',
                'Theme',
               'Category',
               'Kickstarted',
               'ImagePath',
          ]

In [11]:
game_ids_current = pd.read_pickle('data_cleaned/game_ids_current.pkl')
game_ids = list(game_ids_current)

In [ ]:
start_position =0
end_position = 1000
file_suffix = 0

overall_start = time.time()
while end_position < (len(game_ids)+1):
    
    games = pd.DataFrame(columns=columns)
    designers = pd.DataFrame(columns=['BGGId'])
    categories = pd.DataFrame(columns=['BGGId'])
    mechanics = pd.DataFrame(columns=['BGGId'])
    artists = pd.DataFrame(columns=['BGGId'])
    publishers = pd.DataFrame(columns=['BGGId'])
    subcategories = pd.DataFrame(columns=['BGGId'])
    comments = pd.DataFrame(columns=['BGGId'])
    
    ##### File Setup Section #####
    
    # increment file suffix
    file_suffix += 1
    # get file suffix as string
    suffix_str = str(file_suffix)
    
    # print start and end positions
    print("Getting items "+str(start_position+1)+' through '+str(end_position))
    
    # get list of game ids to grab
    #grab_list = game_ids[0][start_position:end_position]
    grab_list = game_ids[start_position:end_position]
    
    # piece together target string of game ids for BGG 
    targets = ''
    for item in grab_list:
        targets += str(item)+','
    
    # log start time for information retrieval
    start = time.time()# log the start time for this entry   
    
    
    ##### API Call Section #####
    
    # Set up Selenium drivers
    options = webdriver.ChromeOptions() # set up chrome options
    options.add_argument("--headless") # set up chrome options
    time.sleep(1) # wait 1 second
    # establish path with targets
    path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&stats=1&type=boardgame'
    driver = webdriver.Chrome(options=options)# initiate chrome driver with options
    print("New page retrieval. May be waiting for load.")
    driver.get(path)# get path
    # wait until the driver finds the element that we need
    element = WebDriverWait(driver, 180).until(EC.presence_of_all_elements_located((By.ID, 'folder0')))
        
    game_page = BeautifulSoup(driver.page_source) # parse page with beautifulsoup    
    
    # make entry for each game item on page
    game_entries = game_page.find_all('item')
    
    print("Items loaded. Processing.")
    ##### Process Each Game #####
    
    for entry in game_entries:
        ##### Get Game Name, BGGId, and check that game should be included in list #####

        ##### Check is expansion #####
        #gametype = entry['type'] # check game type
        #if gametype != 'boardgame':
        #    continue
        #else: pass
       

        # check that this game has sufficient user ratings to incluide
        try:
            user_ratings = int(entry.find('usersrated')['value'])# get the number of user ratings
        
            if user_ratings < 30: #check if user ratings are under 30
                continue
        except: continue
            
        # get game name and BGG ID
        game_name = entry.find('name', type='primary')['value']
        game_id = entry['id']
        #print("Name: "+game_name+", BGG ID: "+str(game_id))

        
        ##### Get Basic Stats #####

        #print("Getting basic stats")
        description = entry.find('description').text # description text of the game
        
        try:
            year_pub = int(entry.find('yearpublished')['value']) # year published
            if year_pub > 2021:
                continue
        except: pass
            
        try: minplayers = int(entry.find('minplayers')['value']) # minimum players
        except: minplayers = None
            
        try: maxplayers = int(entry.find('maxplayers')['value']) # maximum players
        except: maxplayers = None
            
        avg_rating = float(entry.find('average')['value']) # average rating
        bayes_avg = float(entry.find('bayesaverage')['value']) # bayes average rating
        std_dev = float(entry.find('stddev')['value']) # standard deviation of rating
        num_owned = int(entry.find('owned')['value']) # num of people own this game
        num_want = int(entry.find('wanting')['value']) # num of people want this game
        num_wish = int(entry.find('wishing')['value']) # num of people with game on wishlist
        num_weight_votes = int(entry.find('numweights')['value']) # num of votes for game weight
        game_weight = float(entry.find('averageweight')['value']) # voted game weight
        
        try: image_path = entry.find('image').text # path to image
        except: image_path = None
            
        try:  mfg_play_time = int(entry.find('playingtime')['value']) # mfg stated playtime
        except: mfg_play_time = None
        try: comm_min_play = int(entry.find('minplaytime')['value']) # community min playtime
        except: comm_min_play = None
            
        try: comm_max_play = int(entry.find('maxplaytime')['value']) # community max playtime
        except: comm_max_play = None
        
        try: mfg_age = int(entry.find('minage')['value']) # mfg min age
        except: mfg_age = None
            
        #num_comments = int(entry.find('comments')['totalitems']) # num of ratings comments
        num_alts = len(entry.find_all('name', type='alternate')) # number alternate versions
        num_expansions = len(entry.find_all('link', type='boardgameexpansion')) # number of expansions
        num_implementations = len(entry.find_all('link', type='boardgameimplementation')) # number of implementations    
        

        
        ##### Get reimplementation flag #####
        reimplementation = entry.find('link', type="boardgameimplementation", inbound="true") # check if game is a reimplementation
        if reimplementation: 
            reimplements = 1 # if it's a reimplementation, flag it 1
        else: 
            reimplements = 0
 


        ##### Basic stats requiring some compaction/refinement #####

        # community age min
        try:
            age_poll = entry.find('poll', title="User Suggested Player Age").find_all('result')

            total = 0
            items = 0
        
            for item in age_poll:   
                vote = int(item['numvotes']) * int(item['value'][:2])
                total += vote
                items += int(item['numvotes'])

            if items>0: comm_age = total/items # make sure not dividing by 0, get community recommended age
            else: comm_age=None # if no votes, record none
        except: comm_age=None
        
        # Language Ease
        try:
         
            lang_poll = entry.find('poll', title="Language Dependence").find_all('result')
            total, items = 0, 0

            for item in lang_poll:   
                vote = int(item['numvotes']) * int(item['level'])
                total += vote
                items += int(item['numvotes'])

            if items>0: lang_ease = total/items # make sure not dividing by 0, get community language ease
            else: lang_ease=None # if no votes, record none
        except: lang_ease=None # if no votes, record none
            
        try:    
            # Best and Good Players
            players = entry.find('poll', title="User Suggested Number of Players").find_all('results') # get user players poll
            player_num_votes = int(entry.find('poll', title="User Suggested Number of Players")['totalvotes'])# get total votes
        
            best_players, best_score, good_players = 0, 0, [] # set up for best players loop
        
            if player_num_votes > 30: # evaluate if more than 30 votes for num players
                for player in players:
                    best = int(player.find('result', value='Best')['numvotes'])
                    rec = int(player.find('result', value='Recommended')['numvotes'])
                    score = best*2 + rec*1
                    positives = best+rec
                    ratio = positives/player_num_votes
                    if score > best_score: best_players, best_score = player['numplayers'], score # put in # players for best score
                    if ratio > .5: good_players.append(player['numplayers']) # put in good players if over 50% ratio
            else: best_players=None
        except: best_players=None
        
        ##### Skip dynamic content which cannot be batched #####
            
        #this_game['NumFans']=int(num_fans),
        #this_game['NumPageViews']=int(num_views),
        #this_game['RulesPosts']=int(rules_threads),
        #this_game['TotalPosts']=int(total_threads),            
        #this_game['NumAwards'] = int(num_awards)              
            
        
        # make dataframe for this game
        this_game = pd.DataFrame()
        this_game['BGGId']=int(game_id),
        this_game['Name']=game_name,
        this_game['Description']=description,
        this_game['YearPublished']=int(year_pub),
        this_game['GameWeight']=float(game_weight),
        this_game['AvgRating']=float(avg_rating),
        this_game['BayesAvgRating']=float(bayes_avg),
        this_game['StdDev']=float(std_dev),
        this_game['MinPlayers']=minplayers,
        this_game['MaxPlayers']=maxplayers,
        try: this_game['ComAgeRec']=float(comm_age),
        except: this_game['ComAgeRec']=None,
        try: this_game['LanguageEase']=float(lang_ease),
        except: this_game['LanguageEase']=None,
        this_game['BestPlayers']=best_players,
        this_game['GoodPlayers']=good_players,
        this_game['NumOwned']=int(num_owned),
        this_game['NumWant']=int(num_want),
        this_game['NumWish']=int(num_wish),
        this_game['NumWeightVotes']=int(num_weight_votes),
        this_game['MfgPlaytime']=mfg_play_time,
        this_game['ComMinPlaytime']=comm_min_play,
        this_game['ComMaxPlaytime']=comm_max_play,
        this_game['MfgAgeRec']=mfg_age,
        this_game['NumUserRatings']=int(user_ratings),
        #this_game['NumComments']=int(num_comments),
        this_game['NumAlternates']=int(num_alts),
        this_game['NumExpansions']=int(num_expansions),
        this_game['NumImplementations']=int(num_implementations),
        this_game['IsReimplementation']=int(reimplements),
        this_game['ImagePath']=image_path
            
        
        # add unique information to end of df
        
        # Add game ranks
        ranks = entry.find_all('rank')
        try:
            for item in ranks:
                this_game['Rank:'+item['name']] = float(item['value'])
        except: pass
        
        # Try to add components
        try: 
            families = entry.find_all('link', type='boardgamefamily', value=re.compile("Component"))
            for item in families:                    
                this_game['Components:'+item['name']] = item['value']
        except: pass
            
        
        # Try to add game series/family
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Game:"))['value'].strip('Game:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Series:"))['value'].strip('Series:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            setting = entry.find('link', type='boardgamefamily', value=re.compile("Setting:"))['value'].strip('Setting:').strip(' ')
            this_game['Setting'] = setting
        except: pass
            
        
        # Try to add theme
        try:
            theme = entry.find('link', type='boardgamefamily', value=re.compile("Theme:"))['value'].strip('Theme:').strip(' ')
            this_game['Theme'] = theme
        except: pass
            
        try:
            mechanism = entry.find('link', type='boardgamefamily', value=re.compile("Mechanism:"))['value'].strip('Mechanism:').strip(' ')
            this_game['Mechanism'] = mechanism
        except: pass
        
        # Try to add game category
        try:
            category = entry.find('link', type='boardgamefamily', value=re.compile("Category:"))['value'].strip('Category:').strip(' ')
            this_game['Category'] = category
        except: pass
        
        
        # Try is Kickstarted
        try:
            entry.find('link', type='boardgamefamily', value=re.compile("Crowdfunding"))['value']
            this_game['Kickstarted'] = int(1)
        except: pass
     
    
    
        ##### Get subcategories #####
        
        all_subcategories = entry.find_all('link', type='boardgamecategory')
        
        categories_hold = pd.DataFrame(columns=['BGGId'])
        subcategory = {'BGGId':int(game_id)}
    
        for item in all_subcategories:
            subcategory[item['value']] = int(1)      
        
        categories_hold = categories_hold.append(subcategory, ignore_index=True)    
        
        
        # create specialty dataframes
        designer = create_designers(entry, game_id)
        category = create_categories(entry, game_id)
        mechanic = create_mechanics(entry, game_id)
        artist = create_artists(entry, game_id)
        publisher = create_publishers(entry, game_id)
            
        games = games.append(this_game, ignore_index = True)
        designers = designers.append(designer, ignore_index=True)
        categories = categories.append(category, ignore_index=True)
        mechanics = mechanics.append(mechanic, ignore_index=True)
        artists = artists.append(artist, ignore_index=True)
        publishers = publishers.append(publisher, ignore_index=True)
        subcategories = subcategories.append(categories_hold, ignore_index=True)
        
    games.to_pickle('data_dirty/pulled_games/games'+suffix_str+'.pkl')
    designers.to_pickle('data_dirty/pulled_games/designers'+suffix_str+'.pkl')
    categories.to_pickle('data_dirty/pulled_games/categories'+suffix_str+'.pkl')
    mechanics.to_pickle('data_dirty/pulled_games/mechanics'+suffix_str+'.pkl')
    artists.to_pickle('data_dirty/pulled_games/artists'+suffix_str+'.pkl')
    publishers.to_pickle('data_dirty/pulled_games/publishers'+suffix_str+'.pkl')
    subcategories.to_pickle('data_dirty/pulled_games/subcategories'+suffix_str+'.pkl')
    
    print("Finished items in this group")
    
    print(f'Time: {time.time() - start}\n\n')       
    
    start_position += 1000
    end_position += 1000
    
print(f'Time: {time.time() - overall_start}\n\n') 